### Contextualização:

Um levantamento foi realizado com o objetivo de investigar a influência de normas culturais e a aplicação da lei no controle da corrupção, focando no comportamento de estacionamento dos diplomatas das Nações Unidas em Manhattan. Até 2002, a imunidade diplomática protegia os diplomatas da ONU de penalidades de estacionamento, deixando suas ações restritas apenas pelas normas culturais. Em 2002, as autoridades adquiriram o direito de confiscar as placas diplomáticas dos infratores.

**Perguntas**:

1. Qual seria a quantidade média esperada de violações de trânsito para um país cujo corpo diplomático seja composto por X1 membros, considerando o período anterior à vigência da lei e cujo índice de corrupção seja igual a X2?

2. Qual seria a quantidade média esperada de violações de trânsito para o mesmo país, porém agora considerando a vigência da lei?

Para isso, serão construídos um <span style="font-family:Comic Sans MS; color:red">modelo preditivo Poisson</span> e um <span style="font-family:Comic Sans MS; color:red">modelo preditivo Binomial Negativo</span> e, por fim, será feita a comparação dos resultados produzidos pelos modelos.

In [3]:
# Instalação dos pacotes

!pip install pandas
!pip install numpy
!pip install -U seaborn
!pip install matplotlib
!pip install plotly
!pip install scipy
!pip install statsmodels
!pip install scikit-learn
!pip install statstests

In [1]:
# Importação dos pacotes

import pandas as pd # manipulação de dados em formato de dataframe
import numpy as np # operações matemáticas
import seaborn as sns # visualização gráfica
import matplotlib.pyplot as plt # visualização gráfica
from math import exp, factorial # funções matemáticas 'exp' e 'factorial'
import statsmodels.api as sm # estimação de modelos
import statsmodels.formula.api as smf # estimação de modelos de contagem
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP,ZeroInflatedPoisson
# pacote acima para a estimação dos modelos ZINB e ZIP, respectivamente
from statsmodels.discrete.discrete_model import NegativeBinomial, Poisson
# pacote anterior para a realização do teste de Vuong
from scipy import stats # estatística chi2
from statsmodels.iolib.summary2 import summary_col # comparação entre modelos

import warnings
warnings.filterwarnings('ignore')

<div style="text-align: center;">
     <strong<p align="center">REGRESSÃO PARA DADOS DE CONTAGEM</po</strong>
</div>


In [5]:
# CARREGAMENTO DA BASE DE DADOS 'corruption'
df_corruption = pd.read_csv('corruption.csv', delimiter=',')
df_corruption

,country,code,violations,staff,post,corruption
0,Angola,AGO,50,9,no,1.047506
1,Angola,AGO,1,9,yes,1.047506
2,Albania,ALB,17,3,no,0.921079
3,Albania,ALB,0,3,yes,0.921079
4,United Arab Emirates,ARE,0,3,no,-0.779468
...,...,...,...,...,...,...
293,Zaire,ZAR,0,6,yes,1.582807
294,Zambia,ZMB,37,9,no,0.562193
295,Zambia,ZMB,0,9,yes,0.562193
296,Zimbabwe,ZWE,43,14,no,0.133379


In [6]:
# Características das variáveis do dataset
df_corruption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     298 non-null    object 
 1   code        298 non-null    object 
 2   violations  298 non-null    int64  
 3   staff       298 non-null    int64  
 4   post        298 non-null    object 
 5   corruption  298 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 14.1+ KB


In [8]:
# Estatísticas univariadas
df_corruption.describe()

,violations,staff,corruption
count,298.000000,298.000000,298.000000
mean,6.496644,11.805369,0.013641
std,18.210377,11.079558,1.012474
min,0.000000,2.000000,-2.582988
25%,0.000000,6.000000,-0.415147
50%,0.000000,9.000000,0.326959
75%,3.000000,14.000000,0.720250
max,167.000000,86.000000,1.582807
